# CORRECT RMS VALUES IN SUMMMARY TABLE AND PG RESULTS

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

We detected that during the calculation of the Generalized Lomb-Scargle periodograms we made a mistake, recording the square of the RMS value (the value of the `gls.\_YY` member variable) instead of the RMS value.

This notebook correct the values in all the summary tables and the result files. Result images are not affected.

## Modules and configuration

### Modules

In [3]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import os
#import time

#from scipy import stats

# https://github.com/mzechmeister/GLS
# from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [41]:
# Configuration:

# Update type (just in case some problem happens in the first run):
ROOT_SQ_GTO = 0 # 0: do not update the RMS values in the GTO table, 1: update the RMS values.
ROOT_SQ_CVIS = 0 # 0: do not update the RMS values in the CARM_VIS table, 1: update the RMS values.

# Files and folders:
GTO_FILE = "../data/GTO_objects_with_PG.csv"
CARM_VIS_FILE = "../data/CARM_VIS_objects_with_PG.csv"
OUT_GTO_FILES = "../data/CARMENES_GTO_RVs_PGs/"
#OUT_CARM_VIS_FILES = "../data/CARMENES_GTO_RVs_PGs/"
#OUT_TESS_FILES = "../data/CARMENES_GTO_TESS_PGs/"


## Correct RMS for original CARMENES GTO data

### GTO data loading

In [11]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_TESS,e_T0_PG_TESS,offset_PG_TESS,e_offset_PG_TESS,FAP_PG_TESS,valid_PG_TESS,error_PG_TESS,elapsed_time_PG_TESS,fits_file_TESS,fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.002199,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000974,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.609498,0.010704,8262.219751,1.365667,1.000000,1.0,NaN,344.002685,../data/CARMENES_GTO_TESS_PGs/J23548+385_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23548+3...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1354.108815,0.001261,7767.134654,0.094298,0.064148,1.0,NaN,473.533042,../data/CARMENES_GTO_TESS_PGs/J23505-095_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23505-0...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000984,NaN,NaN


In [12]:
print(gto.columns.to_list())

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

### Correct the RMS values in the table

In [13]:
gto[['wrms_RV', 'wrms_TESS']].head()

,wrms_RV,wrms_TESS
0,3.893666e+06,NaN
1,1.262657e+05,NaN
2,8.125833e+02,24030.125000
3,1.899900e+01,162.921387
4,1.096774e+01,NaN


In [14]:
if ROOT_SQ_GTO == 1:
    print("Not updating RMS values.")
    gto['wrms_RV'] = gto['wrms_RV'].map(lambda x: np.sqrt(x) if x is not None else x)
    gto['wrms_TESS'] = gto['wrms_TESS'].map(lambda x: np.sqrt(x) if x is not None else x)
else:
    print("Not updating RMS values.")

In [15]:
gto[['wrms_RV', 'wrms_TESS']].head()

,wrms_RV,wrms_TESS
0,1973.237494,NaN
1,355.338886,NaN
2,28.505846,155.016531
3,4.358785,12.764066
4,3.311758,NaN


### Correct the RMS value at the headers of all related files

In [33]:
#for i in range(0, 3): # TEST
for i in range(0, len(gto)):
    # Correct the RMS values in the RV FITS header:
    fits_file_rv = gto.loc[i, 'fits_file_RV']
    if os.path.isfile(str(fits_file_rv)):
        hdul = fits.open(fits_file_rv,
                         mode='update', memmap=False)
        hdul['RV_CURVE'].header['RMSFLUX'] = gto.loc[i, 'wrms_RV']
        hdul.close() # Already saves the changes to the file, no need to do 'HDUList.flush()'
        del hdul
    else:
        print("[i=%d, star=%s, RV] %s is not a file, skipping..." %(i, gto.loc[i, 'Karmn'], fits_file_rv))
    # Correct the RMS values in the TESS FITS header:
    fits_file_tess = gto.loc[i, 'fits_file_TESS']
    if os.path.isfile(str(fits_file_tess)):
        hdul = fits.open(fits_file_tess,
                         mode='update', memmap=False)
        hdul['TESS_LC'].header['RMSFLUX'] = gto.loc[i, 'wrms_TESS']
        hdul.close() # Already saves the changes to the file, no need to do 'HDUList.flush()'
        del hdul
    else:
        print("[i=%d, star=%s, TESS] %s is not a file, skipping..." %(i, gto.loc[i, 'Karmn'], fits_file_tess))


[i=0, star=J23585+076, TESS] nan is not a file, skipping...
[i=1, star=J23556-061, TESS] nan is not a file, skipping...
[i=4, star=J23492+024, TESS] nan is not a file, skipping...
[i=6, star=J23419+441, TESS] nan is not a file, skipping...
[i=8, star=J23351-023, TESS] nan is not a file, skipping...
[i=9, star=J23340+001, TESS] nan is not a file, skipping...
[i=11, star=J23216+172, TESS] nan is not a file, skipping...
[i=12, star=J23113+085, TESS] nan is not a file, skipping...
[i=13, star=J23064-050, TESS] nan is not a file, skipping...
[i=14, star=J22565+165, TESS] nan is not a file, skipping...
[i=15, star=J22559+178, TESS] nan is not a file, skipping...
[i=17, star=J22518+317, TESS] nan is not a file, skipping...
[i=18, star=J22503-070, TESS] nan is not a file, skipping...
[i=20, star=J22330+093, TESS] nan is not a file, skipping...
[i=25, star=J22125+085, TESS] nan is not a file, skipping...
[i=26, star=J22115+184, TESS] nan is not a file, skipping...
[i=29, star=J22096-046, TESS] 

### Store the table result

In [34]:
gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)

## Correct RMS for CARM_VIS (NZS-corrected) data

### CARM_VIS data loading

In [35]:
# Load GTO data table:
cvis = pd.read_csv(CARM_VIS_FILE, sep=',', decimal='.')
cvis.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_TESS,e_T0_PG_TESS,offset_PG_TESS,e_offset_PG_TESS,FAP_PG_TESS,valid_PG_TESS,error_PG_TESS,elapsed_time_PG_TESS,fits_file_TESS,fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.002958,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.609498,0.010704,8262.219751,1.365667,1.000000,1.0,NaN,344.002685,../data/CARMENES_GTO_TESS_PGs/J23548+385_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23548+3...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1354.108815,0.001261,7767.134654,0.094298,0.064148,1.0,NaN,473.533042,../data/CARMENES_GTO_TESS_PGs/J23505-095_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23505-0...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001986,NaN,NaN


In [36]:
print(cvis.columns.to_list())

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

### Correct the RMS values in the table

In [37]:
cvis[['wrms_RV', 'wrms_TESS']].head()

,wrms_RV,wrms_TESS
0,3.982350e+06,NaN
1,1.189797e+05,NaN
2,8.457554e+02,24030.125000
3,1.130713e+01,162.921387
4,4.769276e+00,NaN


In [44]:
if ROOT_SQ_CVIS == 1:
    print("Updating RMS values.")
    cvis['wrms_RV'] = cvis['wrms_RV'].map(lambda x: np.sqrt(x) if x is not None else x)
    gto['wrms_TESS'] = cvis['wrms_TESS'].map(lambda x: np.sqrt(x) if x is not None else x)
else:
    print("Not updating RMS values.")

Updating RMS values.


In [45]:
cvis[['wrms_RV', 'wrms_TESS']].head()

,wrms_RV,wrms_TESS
0,1995.582536,NaN
1,344.934347,NaN
2,29.081874,24030.125000
3,3.362607,162.921387
4,2.183867,NaN


### Correct the RMS value at the headers of all related files

In [47]:
#for i in range(0, 3): # TEST
for i in range(0, len(cvis)):
    # Correct the RMS values in the RV FITS header:
    fits_file_rv = cvis.loc[i, 'fits_file_RV']
    if os.path.isfile(str(fits_file_rv)):
        hdul = fits.open(fits_file_rv,
                         mode='update', memmap=False)
        hdul['RV_CURVE'].header['RMSFLUX'] = cvis.loc[i, 'wrms_RV']
        hdul.close() # Already saves the changes to the file, no need to do 'HDUList.flush()'
        del hdul
    else:
        print("[i=%d, star=%s, RV] %s is not a file, skipping..." %(i, cvis.loc[i, 'Karmn'], fits_file_rv))
    # Correct the RMS values in the TESS FITS header:
    fits_file_tess = cvis.loc[i, 'fits_file_TESS']
    if os.path.isfile(str(fits_file_tess)):
        hdul = fits.open(fits_file_tess,
                         mode='update', memmap=False)
        hdul['TESS_LC'].header['RMSFLUX'] = cvis.loc[i, 'wrms_TESS']
        hdul.close() # Already saves the changes to the file, no need to do 'HDUList.flush()'
        del hdul
    else:
        print("[i=%d, star=%s, TESS] %s is not a file, skipping..." %(i, cvis.loc[i, 'Karmn'], fits_file_tess))


[i=0, star=J23585+076, TESS] nan is not a file, skipping...
[i=1, star=J23556-061, TESS] nan is not a file, skipping...
[i=4, star=J23492+024, TESS] nan is not a file, skipping...
[i=6, star=J23419+441, TESS] nan is not a file, skipping...
[i=8, star=J23351-023, TESS] nan is not a file, skipping...
[i=9, star=J23340+001, TESS] nan is not a file, skipping...
[i=11, star=J23216+172, TESS] nan is not a file, skipping...
[i=12, star=J23113+085, TESS] nan is not a file, skipping...
[i=13, star=J23064-050, TESS] nan is not a file, skipping...
[i=14, star=J22565+165, TESS] nan is not a file, skipping...
[i=15, star=J22559+178, TESS] nan is not a file, skipping...
[i=17, star=J22518+317, TESS] nan is not a file, skipping...
[i=18, star=J22503-070, TESS] nan is not a file, skipping...
[i=20, star=J22330+093, TESS] nan is not a file, skipping...
[i=25, star=J22125+085, TESS] nan is not a file, skipping...
[i=26, star=J22115+184, TESS] nan is not a file, skipping...
[i=29, star=J22096-046, TESS] 

### Store the table result

In [48]:
cvis.to_csv(CARM_VIS_FILE, sep=',', decimal='.', index=False)

# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have fixed the errors in the RMS values, both in the tables and in the _FITS_ files.